In [134]:
#imports
import config
import requests
import json
import csv
import pprint
import pandas as pd
import time
import ast
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import regex as re

# import unicodedata.category as cat
import unicodedata
import sys

# pretty printer for nicer formating when needed
pp = pprint.PrettyPrinter(indent = 1)

In [135]:
anilist = pd.read_excel('anilist_cleaned.xlsx')
tmdb = pd.read_excel('tmdb_cleaned.xlsx')
combined_dataset = pd.read_excel('tmdb_cleaned.xlsx')

In [136]:
anilist_cleaned_name = list(anilist['cleaned_name'])
anilist_common_name = list(anilist['common_name'])
anilist_name = list(anilist['name'])

In [131]:
anilist.head()

,Unnamed: 0,anilist_id,name,original_name,common_name,synonyms,description,origin_country,average_score,mean_score,genres,popularity,favourites,source,is_adult,start_date,year,month,cleaned_name
0,0,1,Cowboy Bebop,カウボーイビバップ,Cowboy Bebop,"['카우보이 비밥', 'קאובוי ביבופ', '카우보이비밥', 'קאובויב...","Enter a world in the distant future, where Bou...",JP,86.0,86.0,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",173360,11908,ORIGINAL,False,1998-04-03,1998.0,4,カウボーイビバップ
1,1,5,Cowboy Bebop: Tengoku no Tobira,カウボーイビバップ天国の扉,Cowboy Bebop: The Movie - Knockin' on Heaven's...,"['Cowboy Bebop Movie', 'Cowboy Bebop: The Movi...","As the Cowboy Bebop crew travels the stars, th...",JP,82.0,82.0,"['Action', 'Drama', 'Mystery', 'Sci-Fi']",34716,612,ORIGINAL,False,2001-09-01,2001.0,9,カウボーイビバップ天国の扉
2,2,6,Trigun,トライガン,Trigun,[],Trigun takes place in the distant future on a ...,JP,79.0,79.0,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...",61248,1984,MANGA,False,1998-04-01,1998.0,4,トライガン
3,3,7,Witch Hunter Robin,ウィッチハンター ロビン,Witch Hunter Robin,"['WHR', 'whr']",Witches are individuals with special powers li...,JP,68.0,68.0,"['Action', 'Drama', 'Mystery', 'Supernatural']",9430,68,ORIGINAL,False,2002-07-02,2002.0,7,ウィッチハンターロビン
4,4,8,Bouken Ou Beet,冒険王ビィト,Beet the Vandel Buster,"['Adventure King Beet', 'adventurekingbeet']",It is the dark century and the people are suff...,JP,62.0,65.0,"['Adventure', 'Fantasy', 'Supernatural']",1379,6,MANGA,False,2004-09-30,2004.0,9,冒険王ビィト


In [137]:
for index, row in combined_dataset.iterrows():
###################################################################################################################
    # checking TMDB's name against Anilist's romaji. Least accurate(because TMDB seldom have the romaji name), placed first so can be overriden
    common_name = row['name']
    if common_name in anilist_name:
        # getting total number of votes from anilist's data
        try:
            anilist_users = anilist.loc[anilist['name'].isin([f'{common_name}'])]['popularity'].values[0]
        except:
            anilist_users = None
        
        combined_dataset.loc[index, 'anilist_users'] = anilist_users

        # getting average score given from anilist's data
        try:
            anilist_score = anilist.loc[anilist['name'].isin([f'{common_name}'])]['mean_score'].values[0]
        except:
            anilist_score = None

        combined_dataset.loc[index, 'anilist_average_score'] = anilist_score

        # getting genres from anilist
        try:
            anilist_genre = anilist.loc[anilist['name'].isin([f'{common_name}'])]['genres'].values[0]
        except:
            anilist_genre = None
        
        combined_dataset.loc[index, 'anilist_genres'] = anilist_genre
        
        # getting source material from anilist
        try:
            anilist_source = anilist.loc[anilist['name'].isin([f'{common_name}'])]['source'].values[0]
        except:
            anilist_source = None
        
        combined_dataset.loc[index, 'source_material'] = anilist_source

        # getting is_adult from anilist
        try:
            anilist_is_adult = anilist.loc[anilist['name'].isin([f'{common_name}'])]['is_adult'].values[0]
        except:
            anilist_is_adult = None

        combined_dataset.loc[index, 'is_adult'] = anilist_is_adult

        # getting anilist id
        try:
            anilist_id = anilist.loc[anilist['name'].isin([f'{common_name}'])]['anilist_id'].values[0]
        except:
            anilist_id = None
        
        combined_dataset.loc[index, 'anilist_id'] = anilist_id

###################################################################################################################
    # checking TMDB's name against Anilist's name. More accurate
    if common_name in anilist_common_name:
        # getting total number of votes from anilist's data
        try:
            anilist_users = anilist.loc[anilist['common_name'].isin([f'{common_name}'])]['popularity'].values[0]
        except:
            anilist_users = None
        
        combined_dataset.loc[index, 'anilist_users'] = anilist_users

        # getting average score given from anilist's data
        try:
            anilist_score = anilist.loc[anilist['common_name'].isin([f'{common_name}'])]['mean_score'].values[0]
        except:
            anilist_score = None

        combined_dataset.loc[index, 'anilist_average_score'] = anilist_score

        # getting genres from anilist
        try:
            anilist_genre = anilist.loc[anilist['common_name'].isin([f'{common_name}'])]['genres'].values[0]
        except:
            anilist_genre = None
        
        combined_dataset.loc[index, 'anilist_genres'] = anilist_genre
        
        # getting source material from anilist
        try:
            anilist_source = anilist.loc[anilist['common_name'].isin([f'{common_name}'])]['source'].values[0]
        except:
            anilist_source = None
        
        combined_dataset.loc[index, 'source_material'] = anilist_source

        # getting is_adult from anilist
        try:
            anilist_is_adult = anilist.loc[anilist['common_name'].isin([f'{common_name}'])]['is_adult'].values[0]
        except:
            anilist_is_adult = None

        combined_dataset.loc[index, 'is_adult'] = anilist_is_adult

        # getting anilist id
        try:
            anilist_id = anilist.loc[anilist['common_name'].isin([f'{common_name}'])]['anilist_id'].values[0]
        except:
            anilist_id = None
        
        combined_dataset.loc[index, 'anilist_id'] = anilist_id

 ###################################################################################################################
    # checking cleaned name against cleaned name. Most accurate
    cleaned_name = row['cleaned_name']

    if cleaned_name in anilist_cleaned_name:
        # getting total number of votes from anilist's data
        try:
            anilist_users = anilist.loc[anilist['cleaned_name'].isin([f'{cleaned_name}'])]['popularity'].values[0]
        except:
            anilist_users = None
        
        combined_dataset.loc[index, 'anilist_users'] = anilist_users

        # getting average score given from anilist's data
        try:
            anilist_score = anilist.loc[anilist['cleaned_name'].isin([f'{cleaned_name}'])]['mean_score'].values[0]
        except:
            anilist_score = None

        combined_dataset.loc[index, 'anilist_average_score'] = anilist_score

        # getting genres from anilist
        try:
            anilist_genre = anilist.loc[anilist['cleaned_name'].isin([f'{cleaned_name}'])]['genres'].values[0]
        except:
            anilist_genre = None
        
        combined_dataset.loc[index, 'anilist_genres'] = anilist_genre
        
        # getting source material from anilist
        try:
            anilist_source = anilist.loc[anilist['cleaned_name'].isin([f'{cleaned_name}'])]['source'].values[0]
        except:
            anilist_source = None
        
        combined_dataset.loc[index, 'source_material'] = anilist_source

        # getting is_adult from anilist
        try:
            anilist_is_adult = anilist.loc[anilist['cleaned_name'].isin([f'{cleaned_name}'])]['is_adult'].values[0]
        except:
            anilist_is_adult = None

        combined_dataset.loc[index, 'is_adult'] = anilist_is_adult

        # getting anilist id
        try:
            anilist_id = anilist.loc[anilist['cleaned_name'].isin([f'{cleaned_name}'])]['anilist_id'].values[0]
        except:
            anilist_id = None
        
        combined_dataset.loc[index, 'anilist_id'] = anilist_id

In [138]:
for index, row in combined_dataset.iterrows():
    # getting ratings from both datasets
    if row['anilist_average_score'] > -1: # checks if nan
        combined_score = row['anilist_average_score'] * row['anilist_users'] + row['vote_average'] * row['vote_count']
        combined_score /= (row['anilist_users'] + row['vote_count'])
    else:
        combined_score = row['vote_average']

    combined_dataset.loc[index, 'combined_average_score'] = combined_score

    # getting view count from both datasets
    if row['anilist_users'] > -1:
        combined_watchers = row['vote_count'] + row['anilist_users']
    else:
        combined_watchers = row['vote_count']
    combined_dataset.loc[index, 'total_fans'] = combined_watchers

In [139]:
combined_dataset = combined_dataset.drop(columns = ['Unnamed: 0', 'cleaned_name', 'genre_ids'])

In [140]:
combined_dataset = combined_dataset.rename(columns = {'id' : 'tmdb_id', 'vote_count' : 'tmdb_users', 'popularity' : 'tmdb_popularity'})

In [144]:
reordered_columns = ['name', 'overview', 'anilist_genres', 'origin_country', 'original_language', 'original_name', 'first_air_date', 'year', 'total_fans', 'combined_average_score', 'tmdb_popularity',  'tmdb_users',  'vote_average', 'anilist_users', 'anilist_average_score', 'source_material',  'is_adult', 'tmdb_id', 'anilist_id']
combined_dataset = combined_dataset.reindex(columns = reordered_columns)

In [145]:
combined_dataset.to_excel('combined_dataset.xlsx')
combined_dataset.to_csv('combined_dataset.csv')